[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain-academy/blob/main/module-0/basics.ipynb) [![Open in LangChain Academy](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66e9eba12c7b7688aa3dbb5e_LCA-badge-green.svg)](https://academy.langchain.com/courses/take/intro-to-langgraph/lessons/56295530-getting-set-up-video-guide)

# LangChain Academy

Welcome to LangChain Academy! 

## Context

At LangChain, we aim to make it easy to build LLM applications. One type of LLM application you can build is an agent. There’s a lot of excitement around building agents because they can automate a wide range of tasks that were previously impossible. 

In practice though, it is incredibly difficult to build systems that reliably execute on these tasks. As we’ve worked with our users to put agents into production, we’ve learned that more control is often necessary. You might need an agent to always call a specific tool first or use different prompts based on its state. 

To tackle this problem, we’ve built [LangGraph](https://docs.langchain.com/oss/python/langgraph/overview) — a framework for building agent and multi-agent applications. Separate from the LangChain package, LangGraph’s core design philosophy is to help developers add better precision and control into agent workflows, suitable for the complexity of real-world systems.

## Course Structure

The course is structured as a set of modules, with each module focused on a particular theme related to LangGraph. You will see a folder for each module, which contains a series of notebooks. A video will accompany each notebook to help walk through the concepts, but the notebooks are also stand-alone, meaning that they contain explanations and can be viewed independently of the videos. Each module folder also contains a `studio` folder, which contains a set of graphs that can be loaded into [LangSmith Studio](https://docs.langchain.com/langsmith/quick-start-studio), our IDE for building LangGraph applications.

## Setup

Before you begin, please follow the instructions in the `README` to create an environment and install dependencies.

## Chat models

In this course, we'll use Chat Models, which take a sequence of messages as input and return messages as output. LangChain supports many models via [third-party integrations](https://docs.langchain.com/oss/python/integrations/chat). By default, the course will use  [ChatOpenAI](https://docs.langchain.com/oss/python/integrations/chat/openai) because it is both popular and performant. As noted, please ensure that you have an `OPENAI_API_KEY`.

Let's check that your `OPENAI_API_KEY` is set and, if not, you will be asked to enter it.

[Here](https://docs.langchain.com/oss/python/langchain/models) is a useful how-to for all the things that you can do with chat models, but we'll show a few highlights below. If you've run `pip install -r requirements.txt` as noted in the README, then you've installed the `langchain-openai` package. With this, we can instantiate our `ChatOpenAI` model object. You can see pricing for various models [here](https://openai.com/api/pricing/). The notebooks will default to `gpt-4o` because it offers a good balance of quality, price, and speed, but you can also opt for the lower-priced `gpt-3.5` series or more recent models.

There are [a few standard parameters](https://docs.langchain.com/oss/python/langchain/models#parameters) that we can set with chat models. Two of the most common are:

* `model`: the name of the model
* `temperature`: the sampling temperature

`Temperature` controls the randomness or creativity of the model's output where low temperature (close to 0) is more deterministic and focused outputs. This is good for tasks requiring accuracy or factual responses. High temperature (close to 1) is good for creative tasks or generating varied responses. 

In [3]:
from langchain_ollama import ChatOllama
granite_chat = ChatOllama(model="granite3.3:8b", temperature=0)
qwen3_chat = ChatOllama(model="qwen3:8b", temperature=0)

Chat models in LangChain have a number of [default methods](https://reference.langchain.com/python/langchain_core/runnables). For the most part, we'll be using:

* [stream](https://docs.langchain.com/oss/python/langchain/models#stream): stream back chunks of the response
* [invoke](https://docs.langchain.com/oss/python/langchain/models#invoke): call the chain on an input

And, as mentioned, chat models take [messages](https://docs.langchain.com/oss/python/langchain/messages) as input. Messages have a role (that describes who is saying the message) and a content property. We'll be talking a lot more about this later, but here let's just show the basics.

In [4]:
from langchain_core.messages import HumanMessage

# Create a message
msg = HumanMessage(content="Hello world", name="Lance")

# Message list
messages = [msg]

# Invoke the model with a list of messages 
granite_chat.invoke(messages)

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={'model': 'granite3.3:8b', 'created_at': '2026-01-17T19:58:26.535762641Z', 'done': True, 'done_reason': 'stop', 'total_duration': 89737877167, 'load_duration': 43062936183, 'prompt_eval_count': 45, 'prompt_eval_duration': 27409851686, 'eval_count': 10, 'eval_duration': 18976483714, 'logprobs': None, 'model_name': 'granite3.3:8b', 'model_provider': 'ollama'}, id='lc_run--019bcd88-3231-74c0-afae-cc3706e4a1bd-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 45, 'output_tokens': 10, 'total_tokens': 55})

In [5]:
# Invoke the model with a list of messages 
qwen3_chat.invoke(messages)

AIMessage(content='Hello! 🌟 How can I assist you today? 😊', additional_kwargs={}, response_metadata={'model': 'qwen3:8b', 'created_at': '2026-01-17T20:02:09.296571114Z', 'done': True, 'done_reason': 'stop', 'total_duration': 215179801893, 'load_duration': 161723428483, 'prompt_eval_count': 12, 'prompt_eval_duration': 11747631671, 'eval_count': 79, 'eval_duration': 41085603542, 'logprobs': None, 'model_name': 'qwen3:8b', 'model_provider': 'ollama'}, id='lc_run--019bcd89-9efd-7740-b79b-088a0f43890c-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 12, 'output_tokens': 79, 'total_tokens': 91})

We get an `AIMessage` response. Also, note that we can just invoke a chat model with a string. When a string is passed in as input, it is converted to a `HumanMessage` and then passed to the underlying model.


In [6]:
granite_chat.invoke("hello world")

AIMessage(content="Hello! It seems like you're just greeting me. How can I assist you today with information or problem-solving?", additional_kwargs={}, response_metadata={'model': 'granite3.3:8b', 'created_at': '2026-01-17T20:04:59.539110112Z', 'done': True, 'done_reason': 'stop', 'total_duration': 162789294895, 'load_duration': 914872653, 'prompt_eval_count': 45, 'prompt_eval_duration': 147045412317, 'eval_count': 27, 'eval_duration': 8973753397, 'logprobs': None, 'model_name': 'granite3.3:8b', 'model_provider': 'ollama'}, id='lc_run--019bcd8d-0926-7b42-b0b6-13d90e192024-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 45, 'output_tokens': 27, 'total_tokens': 72})

In [7]:
qwen3_chat.invoke("hello world")

AIMessage(content='Hello! 🌟 How can I assist you today? 😊', additional_kwargs={}, response_metadata={'model': 'qwen3:8b', 'created_at': '2026-01-17T20:05:28.216877405Z', 'done': True, 'done_reason': 'stop', 'total_duration': 27703349967, 'load_duration': 892818010, 'prompt_eval_count': 12, 'prompt_eval_duration': 1361395450, 'eval_count': 80, 'eval_duration': 25325512015, 'logprobs': None, 'model_name': 'qwen3:8b', 'model_provider': 'ollama'}, id='lc_run--019bcd8f-9c1f-78e3-af68-ce73789d103b-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 12, 'output_tokens': 80, 'total_tokens': 92})

The interface is consistent across all chat models and models are typically initialized once at the start up each notebooks. 

So, you can easily switch between models without changing the downstream code if you have strong preference for another provider.


## Search Tools

You'll also see [Tavily](https://tavily.com/) in the README, which is a search engine optimized for LLMs and RAG, aimed at efficient, quick, and persistent search results. As mentioned, it's easy to sign up and offers a generous free tier. Some lessons (in Module 4) will use Tavily by default but, of course, other search tools can be used if you want to modify the code for yourself.

In [9]:
from dotenv import load_dotenv
load_dotenv()

True

In [10]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="langchain_tavily")
from langchain_tavily import TavilySearch  # updated at 1.0

tavily_search = TavilySearch(max_results=3)

data = tavily_search.invoke({"query": "What is LangGraph?"})
search_docs = data.get("results", data)

In [11]:
search_docs

[{'url': 'https://huggingface.co/learn/agents-course/en/unit2/langgraph/when_to_use_langgraph',
  'title': 'What is LangGraph ? - Hugging Face Agents Course',
  'content': '# What is LangGraph ? `LangGraph` is a framework developed by LangChain **to manage the control flow of applications that integrate an LLM**. ## When should I use LangGraph ? `LangGraph` is on the other end of the spectrum, it shines when you need **“Control”** on the execution of your agent. LangGraph is particularly valuable when you need **Control over your applications**. It gives you the tools to build an application that follows a predictable process while still leveraging the power of LLMs. Put simply, if your application involves a series of steps that need to be orchestrated in a specific way, with decisions being made at each junction point, **LangGraph provides the structure you need**. ## How does LangGraph work? At its core, `LangGraph` uses a directed graph structure to define the flow of your applicat